In [1]:
from IPython.display import display
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, accuracy_score

import torch


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat')
df = source.read()

In [3]:
target = source.metadata['categories']['multi'] # multi / trinary
labels = df[target].astype('category').cat.codes
# labels = df[target].to_numpy()

In [4]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [7]:
class Conv2DModelCreator:        
    def split_and_preprocess(self, y, X, random_state=42):
        """Split and max scale."""
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=random_state)
        
        train_band_max_values = [X_train[...,i].max() for i in range(X_train.shape[-1])]
        X_train = 2*X_train/train_band_max_values - 1

        test_band_max_values = [X_test[...,i].max() for i in range(X_test.shape[-1])]
        X_test = 2*X_test/test_band_max_values - 1

        return X_train, X_test, y_train, y_test    
        
    def build_model(self, num_classes, input_size):
        model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
        # or any of these variants
        # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
        # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
        # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
        # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
        model
        
    def run(self, labels, filepaths, overwrite=False):
        scores = []
        
        for filepath in tqdm(filepaths, leave=False):
            with open(filepath, 'rb') as f:
                X = np.load(f)

            X_train, X_test, y_train, y_test = self.split_and_preprocess(labels, X)

            model_name = f'conv2d_{filepath.parent}_{filepath.stem}.keras'
            model_path = Path('models').joinpath(model_name)

            if model_path.is_file and not overwrite:
                model = tf.keras.models.load_model(model_path)
            else:
                model = self.build_model(np.unique(labels).size, X_train.shape[1:])
                model.fit(X_train, y_train, epochs=10, verbose=1)
                model.save(model_path)
                
            score = model.evaluate(X_test, y_test, verbose=0)
            scores.append(score[1])
            
        return scores

In [8]:
seasons = ['Spring', 'Summer', 'Autumn', 'Winter']
seasonal_scores = []
for season in tqdm(seasons):
    filepath = sorted(list(Path('seasonal_median').glob(f'{season}.npy')))
    score = Conv2DModelCreator().run(labels, filepath, overwrite=True)
    seasonal_scores.extend(score)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Using cache found in /home/pj/.cache/torch/hub/pytorch_vision_v0.10.0


TypeError: 'NoneType' object is not callable

In [ ]:
traces = [go.Scatter(x=seasons, y=seasonal_scores)]
go.Figure(
    data=traces,
    layout={
        "xaxis": {"title": "Season"},
        "yaxis": {"title": "Accuracy"},
        "title": "Conv2d accuracies"}
)